# Chromadvisor

## 1. Find functional groups function
The goal of the **<font color="blue">find_functionnal_groups</font>** function is to take the smiles of  the molecule to analyze as input and draw out the different functional groups of the molecule, as well as the number of times it is present.

First, we wrote a function capable to detect the functional groups.

We had to put the **<font color="green">output if the input is incorrect</font>**  : "Error : Impossible to convert the SMILES into a molecule."

We then had to choose a **<font color="green">database for the functional groups</font>**> that can be detected. For each functional groups in the database found, we decided whether we wanted to keep them so they could be detected. We deleted those we found too complicated and those we didn’t see in our organic course.


<p style="color:red;">Texte rouge</p>

<span style="color:green; font-weight:bold;">First</span>

<span style="color:blue; font-weight:bold;">find_functionnal_groups</span>

**<font color="green">database for the functional groups</font>**>.
